# 이름 : 
# 학번 : 
--------------
## Final Project 

### Project: Painting classification

### Due date: 2021.06.17

### 다음 링크로 접속하여 우선 팀 명을 바꿉니다. ex)20185141_용권순
https://www.kaggle.com/competitions/2022-deep-learning-basics-challenge

#### Data로 가서 Data를 모두 다운 받습니다. 그 다음은 Template code를 따라서 진행하시면 됩니다. 
>##### 최종실행 후  csv파일을 업로드 하시면 public결과가 보여집니다. 

---

* 아래 여러 셀에서 코드를 완성하는 부분을 수행하고, 보고서에 설명을 최대한 자세하게 적어주세요. 기준은 본인이 이해하고 있다는 것을 표현할 수 있는 부분을 모두 적으시면 됩니다.
  
  
> **제출방법**: 
* 보고서에는 코드 캡쳐 첨부이외에도, 각 코드를 작성하는 기반 이론, 방법론과 설명을 작성하세요.
---------------------------

## Final project 규칙

- 학생들 간의 질문은 금지합니다 (교수 및 조교에게 질문하세요)
- 대 원칙은 "내가 이해한 것만 활용하고 작성한다" 입니다
- 사용하는 기법들과 코드는 모두 보고서에 설명되어야하며, 설명이 없는 경우 감점이나 0점 처리합니다
    - 예를 들어서 검색한 끝에 model ensemble이라는 기법이 유용할 것 같아서, 참고하여 성능을 개선한 경우
        1. 참고자료의 출처를 작성할 것
        2. 기반 이론을 설명할 것
        3. 사용한 코드 분석 및 기법을 설명할 것
- 주석이 없는 경우 채점하지 않습니다
- <p style="color:red;">Pretrained network은 사용하지 않습니다. 본 프로젝트에서는, random 초기화한 parameter들을 직접 학습하는 과정만 허용합니다</p>

    - 랜덤 초기화 기법은 특별한 제약이 없습니다
- Competition 순위와 보고서는 각각 5:5로 점수가 반영됩니다.
------------------------

# 주의: 최종적으로 제출한 csv파일이 code에서 돌린 파일과 일치해야합니다. 
> + 기말 프로젝트 제출 시, model의 파라메터가 저장된 pt 파일, 보고서를 같이 제출합니다. 
> + 체점 과정 중 kaggle에 올라간 최종 csv파일과 제출한 모델을 사용하여 학생이 제출한 code로 csv생성 및 비교합니다(수기 변경 방지).
> + kaggle에 올라간 최종 csv파일과 학생이 제출한 code로 생성한 csv파일이 다를 경우 감점 요소가 생길 수 있으니 주의해주시길 바랍니다.

## 제출 하기 전 모든 셀을 정리해서 커널을 다시 실행하는 것을 권장 

---


# 배경 설명

> #### 여러분은 유명한 화가의 작품을 전시하는 유명한 미술관에 취업했습니다. 
>#### 최근 미술관이 대량의 작품을 입수하여, 각 화가들의 전시회를 열려고 합니다.   
>#### 그러나 작품의 수가 너무나 많아서 사람이 분류하기에는 시간이 오래 걸릴것이라고 판단됩니다. 
>#### 이에 여러분은 딥러닝을 사용하여 효율적인 화가별 그림 분류 프로그램을 만들고자 합니다.

# 목표: 
> ### 본 프로젝트의 완성본은, 새로운 그림(Image)입력하면 <p style="color:red">해당 그림을 그린 화가를 예측합니다.</p>



-------------------------------
------------------------------
-----------------------------

## Step 0: Import library

- 필요하다 생각되는 라이브러리를 미리 import합니다. (참고)
- 필요 없는 라이브러리를 제거하거나 필요한 라이브러리를 추가하셔도 됩니다.

In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange
import random
import pandas as pd
from PIL import Image
from pathlib import Path
from collections import OrderedDict
from time import time, ctime, localtime
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset,DataLoader

import torchvision
import torchvision.transforms as transforms

from torchvision import datasets
from torchvision import models


from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split,train_test_split
from sklearn.model_selection import StratifiedKFold,  KFold


-----------------------------------

## Step 1: Preparing Dataset 
### 1-1: 경로 설정

In [2]:
#본인 컴퓨터에 맞는 train folder의 위치를 입력합니다.(string)
train_dir = '../input/2022-deep-learning-basics-challenge/painting_resize_256/train'   

In [3]:
# 폴더의 목록(화가의 이름)을 가지고 옵니다. 총 class : 15개  
classes = []
#class목록을 저장할 list 
for i in range(len(glob.glob(train_dir+'/*'))):
    classes.append(glob.glob(train_dir+'/*')[i].split('/')[-1])
class_num = len(classes)
print('Total Class num: ',class_num)
print('Class label:')
for i in range(class_num):print('{:5d}th : ' .format(i+1),classes[i])
#glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다.
#기존 양식 코드 그대로 사용.

### 1-2. 제대로 read 되는지 확인 (Transform적용)

In [4]:
classes = sorted(classes)
classes

In [5]:
def check_Image(image_path,transform =None):
    image = Image.open(image_path).convert("RGB") #해당 경로의 image를 RGB순서로 변환
    if transform ==None: #transform이 없으면 그대로 image 반환
        return image
    else: return transform(image) #transform이 있으면 적용 후 image 반환

In [6]:
image = "../input/2022-deep-learning-basics-challenge/painting_resize_256/train/Andy_Warhol/Andy_Warhol_1.jpg"
check_Image(image)

In [7]:
#image가 어떤 식으로 transform 되는지 확인하고 싶을 때 사용하시면 됩니다. 
# transform = transforms.Compose([transforms.RandomEqualize()])
transform = transforms.Compose([transforms.RandomPosterize(bits=2, p=0.5)])
check_Image(image,transform)
#,transforms.RandomEqualize(p=0.5)
#  transforms.RandomPosterize(bits=2, p=0.5),
#                                       transforms.RandomEqualize(p=0.5), #주어진 확률로 무작위로 주어진 이미지의 히스토그램을 균등화한다.

In [8]:
transform = transforms.Compose([transforms.Grayscale(1)])
check_Image(image,transform)
#transform이 제대로 적용되는지 확인한다.

## Step2: Generating DataLoader for Dataset

In [9]:
#데이터 셋에 대한 평균과 표준편차 구하기
data_dir = "../input/2022-deep-learning-basics-challenge/painting_resize_256" #데이터 셋 경로
train_ds_transform = transforms.Compose([transforms.ToTensor()]) #데이터를 오직 텐서화만 진행할 예정
train_ds = datasets.ImageFolder(data_dir+'/train', transform = train_ds_transform) #데이터 셋 텐서화 진행


meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in train_ds] 
#train_ds에 x(이미지데이터)만 사용 _는 lable이라 불필요하다.
#numpy함수를 이용하여 평균을 구한다. train_ds는 채널을 가진 3차원이다. axis를 튜플로 줄 경수 순서대로 진행한다.
#행에 따라 합한뒤 열에따라 합한 후 평균을 구한다.
stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in train_ds]
#표준편차도 평균과 동일하게 구한다.


meanR = np.mean([m[0] for m in meanRGB])
#0번째 index에는 R에 대한 값이 들어있고 R의 평균값을 구한다.
meanG = np.mean([m[1] for m in meanRGB])
#1번째 index에는 G에 대한 값이 들어있고 R의 평균값을 구한다.
meanB = np.mean([m[2] for m in meanRGB])
#2번째 index에는 B에 대한 값이 들어있고 R의 평균값을 구한다.

stdR = np.mean([s[0] for s in stdRGB])
stdG = np.mean([s[1] for s in stdRGB])
stdB = np.mean([s[2] for s in stdRGB])
#평균과 설명이 동일하다.

#데이터 전체에 대한 일반화된 값을 구해야 하므로 평균과 표준편차도 각각에 대한 평균으로 구한다.


print(meanR, meanG, meanB)
print(stdR, stdG, stdB)
# 0.5200497 0.4652539 0.39320767
# 0.20736203 0.19569875 0.1818195

In [10]:
#model_A 에 관한 데이터 셋
data_dir = "../input/2022-deep-learning-basics-challenge/painting_resize_256"
#transforms.RandomRotation(30),transforms.RandomInvert(p=0.5),#p=0.5
#transforms.RandomRotation(30), #랜덤으로 회전시킨다. 현재 0~30도.
#train_data에 대한 transform
#여러 transform들을 활용하기 위해 Compose를 사용한다.
train_transform_A = transforms.Compose([
                                      transforms.RandomCrop(244), #랜덤한 위치에서 244*244만큼 자른다.
                                      transforms.RandomHorizontalFlip(p=0.5), #주어진 확률값으로 수평으로 뒤집는다. 현재 50%
                                      transforms.RandomVerticalFlip(p=0.5), #주어진 확률값으로 수직으로 뒤집는다. 현재 50%
                                      transforms.ToTensor(), #데이터 텐서화.
                                      transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])]) #텐서 이미지 정규화.
                                      #input[channel] = (input[channel] - mean[channel]) / std[channel]공식에 의해 RGB가 normalize된다.

#valid_data에 대한 transform
valid_transform_A = transforms.Compose([
                                      transforms.CenterCrop(244), #가운데에서 244*244만큼 자른다
                                      transforms.ToTensor(),#데이터 텐서화.
                                      transforms.Normalize([meanR, meanG, meanB ], [stdR, stdG, stdB])])#텐서 이미지 정규화.





#폴더 구조를 가지고 있는 데이터셋을 불러올 때 ImageFolder를 사용한다.
#transform에는 내가 만든 transfrom을 대입해 이미지에 변화를 줄 수 있다.
#해당 경로의 ImageFolder data에 transform을 적용한다.
train_data_A = datasets.ImageFolder(data_dir+'/train', transform = train_transform_A)
valid_data_A = datasets.ImageFolder(data_dir+'/train', transform = valid_transform_A)



val_size = 0.2 #valid_data의 크기를 20%로 설정
num_train = len(train_data_A) #train_data_A에 길이(크기) 구하기
num_train = int(num_train) #int로 형변환한다. 소수점은 버린다.
indices = list(range(num_train)) #길이만큼 index부여하면서 list생성한다. indices = [0 ~ int(num_train)-1]
np.random.shuffle(indices) #indices를 랜덤으로 섞는다.
split = int(np.floor(val_size * num_train)) # np.floor은 내림함수이다. 8:2로 나누기 위해 구간을 정한다.
train_idx, valid_idx = indices[split:], indices[:split] #train_dix는 20~99, valide_idx는 0~19만큼 가진다.

#SubsetRandomSampler 주어진 인덱스 목록에서 요소를 무작위로 샘플링
#dataset의 일정 부분을 나눠서 사용할 수 있다.
train_sampler_A = SubsetRandomSampler(train_idx)
valid_sampler_A = SubsetRandomSampler(valid_idx)

train_loader_A = torch.utils.data.DataLoader(train_data_A, batch_size=16, sampler = train_sampler_A)
valid_loader_A = torch.utils.data.DataLoader(valid_data_A, batch_size=16, sampler = valid_sampler_A)
#dataset=train(valid)_data_A, batch_size=16, samper = train(valid)_sampler_A


print(len(train_loader_A))
print(len(valid_loader_A))
#train_loader_A와 valid_loader_A를 batch_size로 구분 했을 때의 길이를 출력해본다.


In [11]:
#model_B
data_dir = "../input/2022-deep-learning-basics-challenge/painting_resize_256"

train_transform_B = transforms.Compose([
                                      transforms.CenterCrop(244), #가운데에서 244*244로 자른다.
                                      transforms.RandomHorizontalFlip(p=0.5), #주어진 확률로 수평으로 뒤집는다.
                                      transforms.RandomRotation(90),#임의의 각도로 이미지를 회전한다. (0~90)
                                      transforms.ToTensor(), #데이터 텐서화
                                      transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])])#텐서 이미지 정규화
#transforms.Resize(255),
valid_transform_B = transforms.Compose([
                                      transforms.CenterCrop(244),#가운데에서 244*244로 자른다.
                                      transforms.ToTensor(),#데이터 텐서화
                                      transforms.Normalize([meanR, meanG, meanB ], [stdR, stdG, stdB])])#텐서 이미지 정규화


#폴더 구조를 가지고 있는 데이터셋을 불러올 때 ImageFolder를 사용한다.
#transform에는 내가 만든 transfrom을 대입해 이미지에 변화를 줄 수 있다.
#해당 경로의 ImageFolder data에 transform을 적용한다.
train_data_B = datasets.ImageFolder(data_dir+'/train', transform = train_transform_B)
valid_data_B = datasets.ImageFolder(data_dir+'/train', transform = valid_transform_B)


val_size = 0.2 #valid_data의 크기를 20%로 설정
num_train = len(train_data_B) #train_data_B에 길이(크기) 구하기
num_train = int(num_train) #int로 형변환한다. 소수점은 버린다.
indices = list(range(num_train)) #길이만큼 index부여하면서 list생성한다. indices = [0 ~ int(num_train)-1]
np.random.shuffle(indices) #indices를 랜덤으로 섞는다.
split = int(np.floor(val_size * num_train)) # np.floor은 내림함수이다. 8:2로 나누기 위해 구간을 정한다.
train_idx, valid_idx = indices[split:], indices[:split] #train_dix는 20~99, valide_idx는 0~19만큼 가진다.

#SubsetRandomSampler 주어진 인덱스 목록에서 요소를 무작위로 샘플링
#dataset의 일정 부분을 나눠서 사용할 수 있다.
train_sampler_B = SubsetRandomSampler(train_idx)
valid_sampler_B = SubsetRandomSampler(valid_idx)


train_loader_B = torch.utils.data.DataLoader(train_data_B, batch_size=16, sampler = train_sampler_B)
valid_loader_B = torch.utils.data.DataLoader(valid_data_B, batch_size=16, sampler = valid_sampler_B)
#dataset=train(valid)_data_B, batch_size=16, samper = train(valid)_sampler_B


print(len(train_loader_B))
print(len(valid_loader_B))
#train_loader_B와 valid_loader_B를 batch_size로 구분 했을 때의 길이를 출력해본다.

In [12]:
#model_C
data_dir = "../input/2022-deep-learning-basics-challenge/painting_resize_256"
#transforms.RandomRotation(30),transforms.RandomInvert(p=0.5),#p=0.5
#transforms.RandomEqualize(p=0.5), #주어진 확률로 무작위로 주어진 이미지의 히스토그램을 균등화한다.
# transforms.RandomAdjustSharpness(sharpness_factor=2),#주어진 이미지의 선명도를 무작위로 조정한다. sharpness_factor는 선명도를 조절항 정도이다. 음수는 불가능.
train_transform_C = transforms.Compose([
                                      transforms.CenterCrop(244),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomEqualize(),
                                      transforms.ToTensor(),#데이터 텐서화
                                      transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])])#텐서 이미지 정규화

valid_transform_C = transforms.Compose([
                                      transforms.CenterCrop(244),#가운데에서 244*244로 자른다.
                                      transforms.ToTensor(),#데이터 텐서화
                                      transforms.Normalize([meanR, meanG, meanB ], [stdR, stdG, stdB])])#텐서 이미지 정규화



#폴더 구조를 가지고 있는 데이터셋을 불러올 때 ImageFolder를 사용한다.
#transform에는 내가 만든 transfrom을 대입해 이미지에 변화를 줄 수 있다.
#해당 경로의 ImageFolder data에 transform을 적용한다.
train_data_C = datasets.ImageFolder(data_dir+'/train', transform = train_transform_C)
valid_data_C = datasets.ImageFolder(data_dir+'/train', transform = valid_transform_C)


val_size = 0.2 #valid_data의 크기를 20%로 설정
num_train = len(train_data_C) #train_data_B에 길이(크기) 구하기
num_train = int(num_train) #int로 형변환한다. 소수점은 버린다.
indices = list(range(num_train)) #길이만큼 index부여하면서 list생성한다. indices = [0 ~ int(num_train)-1]
np.random.shuffle(indices) #indices를 랜덤으로 섞는다.
split = int(np.floor(val_size * num_train)) # np.floor은 내림함수이다. 8:2로 나누기 위해 구간을 정한다.
train_idx, valid_idx = indices[split:], indices[:split] #train_dix는 20~99, valide_idx는 0~19만큼 가진다.


#SubsetRandomSampler 주어진 인덱스 목록에서 요소를 무작위로 샘플링
#dataset의 일정 부분을 나눠서 사용할 수 있다.
train_sampler_C = SubsetRandomSampler(train_idx)
valid_sampler_C = SubsetRandomSampler(valid_idx)


train_loader_C = torch.utils.data.DataLoader(train_data_C, batch_size=16, sampler = train_sampler_C)
valid_loader_C = torch.utils.data.DataLoader(valid_data_C, batch_size=16, sampler = valid_sampler_C)
#dataset=train(valid)_data_C, batch_size=16, samper = train(valid)_sampler_C


print(len(train_loader_C))
print(len(valid_loader_C))
#train_loader_B와 valid_loader_B를 batch_size로 구분 했을 때의 길이를 출력해본다.

--------------------------

## Step 3: Generating Neural Network(model) 
 - <p style="color:red;">Pretrained network은 사용하지 않습니다. 본 프로젝트에서는, random 초기화한 parameter들을 직접 학습하는 과정만 허용합니다</p>
 - 모델을 선정한 이유를 간략하게 정리합니다.

In [13]:
class GNN(nn.Module):
    def __init__(self):
        super().__init__()
        #input  3*244*244
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride = 1, padding=1)
        #input채널이 3, output채널을 64로 설정.  kernel_size=3으로 설정. 필터(kerndel) 이동거리는 1(stride=1)
        #padding=1로 원래 채널의 테두리에 한칸이 늘어나며 값은 0으로 들어간다.
        self.conv1_bn = nn.BatchNorm2d(64)
        #2d배치 정규화로 output 채널값을 num_features에 받는다.
        #더 쉽게 train가능하게 해주며 gradient의 흐름을 향상시킨다.
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride = 1, padding=1)
        self.conv2_bn = nn.BatchNorm2d(128)
        
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride = 1, padding=1)
        self.conv3_bn = nn.BatchNorm2d(256)
        
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride = 1, padding=1)
        self.conv4_bn = nn.BatchNorm2d(512)
        
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, stride = 1, padding=1)
        self.conv5_bn = nn.BatchNorm2d(1024)
        
        self.conv6 = nn.Conv2d(1024, 2048, kernel_size=3, stride = 1, padding=1)
        self.conv6_bn = nn.BatchNorm2d(2048)
        
        
        
        self.pool = nn.MaxPool2d((2,2) , stride=2) 
        #kernel_size = (2,2)로 해당 필터 안에 최대 값을 가져온다.
        #stride =2로 채널의 이동거리이다. MaxPool2d를 이와 같이 설정하면 한 층이 지나면 출력값이 1/2로 줄어든다.
        
        self.fc1 = nn.Linear(2048*3*3, 512)
        self.fc1_bn = nn.BatchNorm1d(512)
        #1d배치 정규화로 output값을 num_feature에 받는다.
        
        self.fc2 = nn.Linear(512, 15)
        
        self.dropout = nn.Dropout(0.5) 
        #신경망의 과적합을 줄이기 위한 대표적인 정규화 기술. 뉴런의 일부를 0으로대치하여 계산에서 제외한다.
        #0.2~0.5가 통상 
        
        self.relu = nn.ReLU(True)
        #max(0,x)를 의미하는 함수로 0보다 작으면 0이 되는 특징을 가지고 있다.
        #0이하의 입력에 대해 0을 출력함으로써 부분적으로 활성화가 가능하며 선형함수이므로 미분 계산이 매우 편리하다.
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool(self.relu(self.conv2_bn(self.conv2(x))))

        x = self.pool(self.relu(self.conv3_bn(self.conv3(x))))
        x = self.pool(self.relu(self.conv4_bn(self.conv4(x))))
        
        x = self.pool(self.relu(self.conv5_bn(self.conv5(x))))
        x = self.pool(self.relu(self.conv6_bn(self.conv6(x))))
        #컨볼루션 -> 배치 정규화 -> activation func -> pooling
        
        x = x.view(-1,2048*3*3)
        #데이터 flatten

        x = self.relu(self.fc1_bn(self.fc1(x)))
        x = self.dropout(x)
        
        x = self.fc2(x)
        
        return x
        

--------------------------

## Step 4: Selecting Cost(Loss) Function & Optimizer
> +  [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) 및 [optimizer](http://pytorch.org/docs/stable/optim.html)를 선택하여 코드를 완성하세요.
  링크에서 다양한 Loss Function과 Optimize Function을 확인 할 수 있습니다

In [14]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

model_A = GNN()

model_B = GNN()

model_C = GNN()
#모델 3개 선언 -> 앙상블 기법을 활용하기 위함.
#스케줄러와 데이터 셋 transform에 차이가 있다.

nSamples = [19,262,210,702,72,
            114,67,233,562,351,
            145,82,269,58,191] #not sort
#각 클래스별 데이터 개수이며 기존 클래스 출력시 나온 순서대로 넣었다.

# nSamples = [262,145,58,562,72,233,19,82,114,191,351,269,67,210,702] #sort


# normedWeights1 = [1 - (x / sum(nSamples)) for x in nSamples] 
#처음에 가중치를 이렇게 조절했는데 적은 데이터를 가지는 클래스와 많은 데이터를 가지는 클래스와 큰 차이가 없었다. 
#이를 토대로 직접 가중치를 설정했다.
# normedWeights1 = [(1 / (x*10)) for x in nSamples] #loss는 적은데 acc도 잘 안오름 최대 약 60정
normedWeights1 = [0.99, 0.7, 0.7, 0.4, 0.97, 
                  0.85, 0.99, 0.7, 0.5, 0.6, 
                  0.8, 0.97, 0.7, 0.99, 0.8]
normedWeights2 = torch.FloatTensor(normedWeights1).to(device)
#가중치를 float형 텐서로 변환했다.
#클래스 불균형이 매우 심하기 때문에 적용했다.



criterion_A = nn.CrossEntropyLoss(normedWeights2)
criterion_B = nn.CrossEntropyLoss(normedWeights2)
criterion_C = nn.CrossEntropyLoss(normedWeights2)

#각 모델에 대한 CrossEntropyLoss이며 동일하게 가중치를 대입했다.



optimizer_A = optim.Adam(model_A.parameters(), lr = 0.0001, weight_decay=0.00001)
scheduler_A = optim.lr_scheduler.MultiStepLR(optimizer_A, milestones=[15,30,45], gamma=0.1, last_epoch=- 1, verbose=True)
#model_A의 파라미터를 받으며 learning rate = 1e-4, weight_decay = 1e-5로 설정했다.
#MultiStepLR learnig rate를 감소시킬 epoch을 정해준다. 감소량은 lr= gamma*lr 
#last_epoch은 학습을 다시 시작할 때 이전에 중단된 스케줄러를 시작하려 하는데 -1이면 스케줄러가 처음부터 시작되었음을 나타낸다.
#verbose = True로 lr의 변동된 값을 확인할 수 있다.

optimizer_B = optim.Adam(model_B.parameters(), lr = 0.0001, weight_decay=0.00001)
#model_B의 파라미터를 받으며 learning rate = 1e-4, weight_decay = 1e-5로 설정했다.
scheduler_B = optim.lr_scheduler.LambdaLR(optimizer=optimizer_B,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=True)
#LambdaLR 초기 learning_rate에 lambda함수에서 나온 값을 곱해서 learning_rate를 계산한다.
#last_epoch은 학습을 다시 시작할 때 이전에 중단된 스케줄러를 시작하려 하는데 -1이면 스케줄러가 처음부터 시작되었음을 나타낸다.
#verbose = True로 lr의 변동된 값을 확인할 수 있다.


optimizer_C = optim.Adam(model_C.parameters(), lr = 0.0001, weight_decay=0.00001)
#model_C의 파라미터를 받으며 learning rate = 1e-4, weight_decay = 1e-5로 설정했다.
scheduler_C = optim.lr_scheduler.MultiStepLR(optimizer_C, milestones=[15,30,45], gamma=0.1, last_epoch=- 1, verbose=True)
#MultiStepLR learnig rate를 감소시킬 epoch을 정해준다. 감소량은 lr= gamma*lr
#last_epoch은 학습을 다시 시작할 때 이전에 중단된 스케줄러를 시작하려 하는데 -1이면 스케줄러가 처음부터 시작되었음을 나타낸다.
#verbose = True로 lr의 변동된 값을 확인할 수 있다.


model_A.to(device)
model_B.to(device)
model_C.to(device)
#각 모델을 cuda에 올린다.
print(device)

--------------------------

## Step 5: Train and validate the configured model
### 학습 및 검증 과정을 진행합니다
+ 학습 과정을 주석으로 설명하세요 
+ Epoch별로 Loss, Accuracy를 출력하여 학습 진행 과정을 확인 할 수 있도록 합니다
+ 출력 예시는 주어지나 정해진 형식은 없습니다
+ 최적의 모델 파라메터를 저장합니다.

For example : 
```python
Started Training...
Epoch: 1    Training Loss: 3.317162     Validation Loss: 4.162958
Epoch: 2    Training Loss: 2.420140     Validation Loss: 4.182362
...
...
Finished training
```

In [15]:
n_epochs = 60 #에폭 수 60

valid_loss_min = np.Inf #valid_loss를 무한으로 주고 내려갈때 마다 저장한다.

train_loss = torch.zeros(n_epochs) #에폭 수 만큼 0으로 채운 텐서 만들기 train_loss를 가질 예정이며 나중에 그래프 그릴 때 사용한다.
valid_loss = torch.zeros(n_epochs) #에폭 수 만큼 0으로 채운 텐서 만들기 valid_loss를 가질 예정이며 나중에 그래프 그릴 때 사용한다.

train_acc = torch.zeros(n_epochs) #에폭 수 만큼 0으로 채운 텐서 만들기 train_acc를 가질 예정이며 나중에 그래프 그릴 때 사용한다.
valid_acc = torch.zeros(n_epochs) #에폭 수 만큼 0으로 채운 텐서 만들기 valid_acc를 가질 예정이며 나중에 그래프 그릴 때 사용한다.


for e in range(0, n_epochs):
        model_A.train() #model_A학습 시작
        for data, labels in train_loader_A:
            data, labels = data.to(device), labels.to(device)
            #data, labels를 cuda로 이동

            optimizer_A.zero_grad()
            #파라미터를 업데이트 한 후 새로운 batch에 대한 새 gradient를 계산해야 하기 때문에 옵티마이저를 초기화한다.
            
            logits = model_A(data) #data를 model_A에 넣어 결과 예측(계산)
            
            loss = criterion_A(logits, labels)#batch loss 계산
            
            loss.backward() #전체 파라미터에 대한 gradient를 구한다.
            optimizer_A.step()  #gradient를 각 파라미터에 적용한다.

            train_loss[e] += loss.item() #loss를 에폭 위치에 넣어준다.

            ps = F.softmax(logits, dim=1) #logits값에 softmax를 취해 0과 1사이의 값으로 바뀐다.
            top_p, top_class = ps.topk(1, dim=1) #주어진 input 텐서의 가장 큰 값 1개를 반환한다.
            equals = top_class == labels.reshape(top_class.shape) #labels의 shape을 top_class의 shape으로 reshape한 후, top_class와 비교한다.
            train_acc[e] += torch.mean(equals.type(torch.float)).item() #True, False를 float형 토치로 변환 후 평균을 에폭 index에 넣어준다.

        train_loss[e] /= len(train_loader_A) #batch_size로 정해진 train_loader_A길이(크기)로 나눈다.
        train_acc[e] /= len(train_loader_A)

        with torch.no_grad():
            model_A.eval() #validation 과정에서 사용하지 않을 layer들을 off(dropout 등)
            for data, labels in valid_loader_A:
                data, labels = data.to(device), labels.to(device)

                logits = model_A(data) #data를 model_A에 넣어 결과 예측(계산)
                loss = criterion_A(logits, labels) #batch loss 계산
                valid_loss[e] += loss.item() #loss를 에폭 위치에 넣어준다.

                ps = F.softmax(logits, dim=1) #logits값에 softmax를 취해 0과 1사이의 값으로 바뀐다.
                top_p, top_class = ps.topk(1, dim=1) #주어진 input 텐서의 가장 큰 값 1개를 반환한다.
                equals = top_class == labels.reshape(top_class.shape) #labels의 shape을 top_class의 shape으로 reshape한 후, top_class와 비교한다.
                valid_acc[e] += torch.mean(equals.type(torch.float)).item() #True, False를 float형 토치로 변환 후 평균을 에폭 index에 넣어준다.

        valid_loss[e] /= len(valid_loader_A) #batch_size로 정해진 valid_loader_A길이(크기)로 나눈다.
        valid_acc[e] /= len(valid_loader_A)
        scheduler_A.step()# 옵티마이저 매개 변수 업데이트(최적화) 단계
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            e, train_loss[e], valid_loss[e]))
        #train_loss와 valid_loss를 epoch마다 출력
        
        print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
            e, train_acc[e], valid_acc[e]))
        #train_acc와 valid_acc를 epoch마다 출력
        
        #loss가 감소하면 state_dict저장. early stopping
        if valid_loss_min > valid_loss[e]:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss[e]))
            validpoint_A = {'model_state_dict': model_A.state_dict(),
                  'optimizer_state_dict': optimizer_A.state_dict()}
            #딕셔너리 타입으로 저장
            torch.save(validpoint_A, 'validpoint_A.pth')

            valid_loss_min = valid_loss[e] #valid_loss 갱신

In [16]:
plt.plot(train_loss, label='train_loss')
plt.plot(valid_loss, label='valid_loss')
plt.legend()
#train_loss와 valid_loss를 그래프로 출력한다.

In [17]:
plt.plot(train_acc, label='train_acc')
plt.plot(valid_acc, label='valid_acc')
plt.legend()
#train_acc와 valid_acc를 그래프로 출력한다.
#train_acc는 계속 상승하는데 valid_acc는 증가하다 점점 감소하다면 그 구간(epoch)부터 overfitting으로 간주한다.
#train_acc와 valid_acc의 간격이 큰 차이가 없으며 성능이 올라간다면 underfitting으로 간주한다.

In [ ]:
n_epochs = 60

valid_loss_min = np.Inf
valid_acc_min = 0

train_loss = torch.zeros(n_epochs)
valid_loss = torch.zeros(n_epochs)

train_acc = torch.zeros(n_epochs)
valid_acc = torch.zeros(n_epochs)


for e in range(0, n_epochs):
        model_B.train()
        for data, labels in train_loader_B:
            data, labels = data.to(device), labels.to(device)

            optimizer_B.zero_grad()

            logits = model_B(data)
            loss = criterion_B(logits, labels)
            loss.backward()
            optimizer_B.step()

            train_loss[e] += loss.item()

            ps = F.softmax(logits, dim=1)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.reshape(top_class.shape)
            train_acc[e] += torch.mean(equals.type(torch.float)).item()

        train_loss[e] /= len(train_loader_B)
        train_acc[e] /= len(train_loader_B)

        with torch.no_grad():
            model_B.eval()
            for data, labels in valid_loader_B:
                data, labels = data.to(device), labels.to(device)

                logits = model_B(data)
                loss = criterion_B(logits, labels)
                valid_loss[e] += loss.item()

                ps = F.softmax(logits, dim=1)
                top_p, top_class = ps.topk(1, dim=1) #logits 이었던거 같은데
                equals = top_class == labels.reshape(top_class.shape)
                valid_acc[e] += torch.mean(equals.type(torch.float)).item()

        valid_loss[e] /= len(valid_loader_B)
        valid_acc[e] /= len(valid_loader_B)
        scheduler_B.step()
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            e, train_loss[e], valid_loss[e]))

        print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
            e, train_acc[e], valid_acc[e]))

        #loss가 감소하면 model저장
        if valid_loss_min > valid_loss[e]:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss[e]))
            validpoint_B = {'model_state_dict': model_B.state_dict(),
                  'optimizer_state_dict': optimizer_B.state_dict()}

            torch.save(validpoint_B, 'validpoint_B.pth')

            valid_loss_min = valid_loss[e]

In [ ]:
plt.plot(train_loss, label='train_loss')
plt.plot(valid_loss, label='valid_loss')
plt.legend()

In [ ]:
plt.plot(train_acc, label='train_acc')
plt.plot(valid_acc, label='valid_acc')
plt.legend()

In [15]:
n_epochs = 60

valid_loss_min = np.Inf
valid_acc_min = 0

train_loss = torch.zeros(n_epochs)
valid_loss = torch.zeros(n_epochs)

train_acc = torch.zeros(n_epochs)
valid_acc = torch.zeros(n_epochs)


for e in range(0, n_epochs):
        model_C.train()
        for data, labels in train_loader_C:
            data, labels = data.to(device), labels.to(device)

            optimizer_C.zero_grad()

            logits = model_C(data)
            loss = criterion_C(logits, labels)
            loss.backward()
            optimizer_C.step()

            train_loss[e] += loss.item()

            ps = F.softmax(logits, dim=1)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.reshape(top_class.shape)
            train_acc[e] += torch.mean(equals.type(torch.float)).item()

        train_loss[e] /= len(train_loader_C)
        train_acc[e] /= len(train_loader_C)

        with torch.no_grad():
            model_C.eval()
            for data, labels in valid_loader_C:
                data, labels = data.to(device), labels.to(device)

                logits = model_C(data)
                loss = criterion_C(logits, labels)
                valid_loss[e] += loss.item()

                ps = F.softmax(logits, dim=1)
                top_p, top_class = ps.topk(1, dim=1) 
                equals = top_class == labels.reshape(top_class.shape)
                valid_acc[e] += torch.mean(equals.type(torch.float)).item()

        valid_loss[e] /= len(valid_loader_C)
        valid_acc[e] /= len(valid_loader_C)
        scheduler_C.step()
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            e, train_loss[e], valid_loss[e]))

        print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
            e, train_acc[e], valid_acc[e]))

        #loss가 감소하면 model저장
        if valid_loss_min > valid_loss[e]:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss[e]))
            validpoint_C = {'model_state_dict': model_C.state_dict(),
                  'optimizer_state_dict': optimizer_C.state_dict()}

            torch.save(validpoint_C, 'validpoint_C.pth')

            valid_loss_min = valid_loss[e]

In [18]:
plt.plot(train_loss, label='train_loss')
plt.plot(valid_loss, label='valid_loss')
plt.legend()

In [19]:
plt.plot(train_acc, label='train_acc')
plt.plot(valid_acc, label='valid_acc')
plt.legend()

--------------------------

## Step 6: Analyze the  training/validation results
+ 위에서 구한 학습 결과를 분석합니다. 
+ 수업에서 배운 그래프를 활용하시는 것을 적극 추천드립니다.
+ 만족할만한 결과가 나올 때 까지 Hyper Parameter Tuning을 진행합니다. 

### Step6을 Step5에서 함께 했습니다.

In [ ]:
# plt.plot(train_loss, label='train_loss')
# plt.plot(valid_loss, label='valid_loss')
# plt.legend()

In [ ]:
# plt.plot(train_acc, label='train_acc')
# plt.plot(valid_acc, label='valid_acc')
# plt.legend()

--------------------------

## Step 7: Define Test Data Set
> Test data를 위한 Custom Test Dataset을 제공해드립니다.   
> 필요한 부분을 채워 사용하시거나 직접 코드를 작성하셔도 됩니다.

In [20]:
'''Custom Dataset을 제작하기 위해서는 3가지를 반드시 포함해야합니다. 
1.def __init__()      :초기화 함수입니다. Dataset을 상속받아 imagepath와 transform을 지정합니다.
2.def __len__()       :dataset의 총 길이를 return하는 함수입니다. 
3.def __getitem__()   :index(idx)에 해당하는 data sample을 return 하는 함수입니다. 
'''
class test_Dataset(Dataset): #test_set을 class로 정의합니다.
    def __init__(self,imgpath,transform=None):
        
        self.imgpath = imgpath
        self.transform = transform
        
    def __len__(self):
        return len(self.imgpath)
    
    def __getitem__(self,idx):
        if self.transform !=None:
            x = self.transform(Image.open(self.imgpath[idx]).convert('RGB'))
        else: x = Image.open(self.imgpath[idx]).convert('RGB')
        return x

In [21]:
#예시 코드입니다. 잘 작동하는지 확인해봅시다.
test_dir = '../input/2022-deep-learning-basics-challenge/painting_resize_256/test'
test_set  = sorted(glob.glob(test_dir+'/*'))
test_data = test_Dataset(test_set)

# 잘 불러오는지 확인해 봅시다. 
print("test_data_len:", test_data.__len__())
test_data.__getitem__(0)

In [22]:
test_dir = '../input/2022-deep-learning-basics-challenge/painting_resize_256/test'
#transforms.Resize(255),
test_transform = torchvision.transforms.Compose([
                                                 transforms.CenterCrop(244),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])])
#test_transform 정의 

test_set  = sorted(glob.glob(test_dir+'/*')) #모든 파일을 읽어서 정리합니다. 
test_data = test_Dataset(test_set,test_transform)

#Data Loader 작성 , (tip: test이므로 Shuffle을 할 필요가 없습니다. )
test_DataLoader = test_loader = torch.utils.data.DataLoader(test_data, batch_size=16)

 

----------------

## Step 8(Final): Predict with Test Data
### Step 8-1: Predict 
+ test data를 우리의 모델로 예측합니다.

In [23]:
#class 재 확인 15개인지 확인하십시오 
print('Total Class num: ',class_num)
print('Class label:')
for i in range(class_num):print('{:5d}th : ' .format(i+1),classes[i])

In [24]:
classes = sorted(classes)
classes

In [26]:
# model_A1과 model_A2는 기존에 성능이 좋았던 model_A와 model_B이다.
model_A1 = GNN() 
model_A1 = torch.load("../input/model-final2/predict_model_A (77.9).pt")
model_A2 = GNN()
model_A2 = torch.load("../input/model-final2/predict_model_B (77.9).pt")

model_A1.to(device)
model_A2.to(device)

model_A1.eval()
model_A2.eval()

#model_C만 성능을 계속 개선시키면서 사용중이므로 load_state_dict를 사용했다.
model_C.load_state_dict(validpoint_C['model_state_dict'])
model_C.eval()

print(device)

In [27]:
pred = []  

#test_loader = iter(test_DataLoader)
# 최종적으로 예측한 각 sample의 label을 담아 놓을 list입니다.  
#0~14까지 우리가 예측한 가장 높은 확률의 label이 들어가면 되겠죠? 
for data in test_DataLoader:
    data = data.to(device)
    logits_A = model_A1(data)
    logits_B = model_A2(data)
    logits_C = model_C(data)
    logits = (logits_A+logits_B+logits_C)/3
    ps = F.softmax(logits, dim=1)
    _, top_class = ps.topk(1, dim=1)
    for i in range(len(data)):
        pred.append(top_class[i].item())
        
#모델 3개가 각각 예측을 한 후 평균 값을 구해서 대입한다.

### Step 8-2 : Making csv file with our predict result
+ csv파일로 만드는 예시를 보여드립니다. 

In [28]:
name=[]                                     # 그림의 Id를 담을 list 생성 
category =[]                                # 그림의 Category를 담을 list 생성
for i in range(test_data.__len__()):        #
    name.append(test_set[i].split('/')[-1]) # 파일 이름에서 label만 가지고 옵니다. 
    category.append(classes[pred[i]])       # pred에 해당하는 class를 category에 집어 넣습니다. 
    
    
tm = localtime()
csv_file_name = f"{tm.tm_mday}day_{tm.tm_hour}h_{tm.tm_min}m_{tm.tm_sec}s"

#파일의 이름은 마음대로 설정할 수 있습니다. 경로를 쓰지 않으면 현재 디렉토리에 알아서 생성됩니다. 
pd.DataFrame({'Id':name,'Category':category}).to_csv(f'predict_{"ff"}.csv',index=False)    

#predict 하기 위해서 model 전체를 저장합니다. 체점을 위해서 필요한 과정입니다.
# torch.save(model_A,f'./predict_{"model_A"}.pt')                                                  
# torch.save(model_B,f'./predict_{"model_B"}.pt')
torch.save(model_C,f'./predict_{"model_C"}.pt')

In [29]:
print(name[:5])
print(category[:5])

In [31]:
# 잘 생성됬는지 확인 , 따로 경로를 설정하지 않았다면,현재 디렉토리에 자동으로 생성됩니다. 
df= pd.read_csv(f'./predict_{"ff"}.csv')
df.head(30) #앞부분 더 확인해 보고싶어서 30으로 늘렸다.

## 생성한 csv파일을 kaggle에 등록하시면 됩니다.

